In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
from crewai import Agent, Task, Crew

In [6]:
import os

os.environ["openai_api_key"] = "ENTER OPENAI API KEY"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = "ENTER SERPER API KEY"

In [7]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [8]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [10]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [11]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [12]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [13]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [14]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [15]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

In [16]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

In [17]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [18]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks and ask questions to ensure we are on track to provide insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: 
{
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques",
    "context": "We need to identify trends and predict market movements for AAPL to provide insights and alerts about significant market opportunities or threats.",
    "coworker": "Data Analyst"
}

> Entering new CrewAgentExecutor chain...
I

In [19]:
from IPython.display import Markdown
Markdown(result)

AAPL Trading Strategies Risk Analysis Report:

Based on the gathered information from various sources, including LinkedIn, Market Chameleon, Barchart.com, Options AI, TradingSim, Investors.com, Saxo Bank, Vestinda, Quantified Strategies, and YouTube, the potential risks associated with the proposed trading strategies for AAPL are as follows:

1. Short Position in Put Option: The strategy of using a short position in a put option with a strike price below the current market price of AAPL shares can expose traders to potential losses if the market moves against their position.

2. Option Trading Strategies: Various option trading strategies such as Straddle, Buy-Write, Risk Reversal, Call Spread, and Covered Calls on Apple can involve risks related to market volatility, stock price movement, and time decay.

3. Day Trading Strategies: Day trading strategies focusing on breakout trading, trend following, scalping, and news-based trading can be risky due to rapid price movements, false breakouts, sudden news events impacting stock prices, and high-frequency trading activity.

4. Technical Analysis Strategies: Strategies based on technical analysis indicators such as moving averages, RSI, volume, and trend identification can be subject to false signals, whipsaws, and market manipulation.

5. Risk Management: Not implementing strict risk management practices, setting appropriate stop-loss orders, and managing position sizes can expose traders to significant losses in volatile trading environments.

Mitigation Strategies:

1. Conduct thorough research and analysis before implementing any trading strategy.
2. Implement risk management practices such as setting stop-loss orders and position sizing based on risk tolerance.
3. Diversify trading strategies to minimize exposure to specific risks and market conditions.
4. Stay informed about the latest news events and developments related to AAPL to adjust trading positions accordingly.
5. Regularly review and adjust trading strategies based on market conditions and performance.

By recognizing these potential risks and implementing the suggested mitigation strategies, traders can navigate the market conditions more effectively and execute trades for AAPL with a higher level of confidence and risk management.